In [4]:
# fingerprint.py
 
import sys
import re
import string
import itertools
import nltk
from stemming.porter2 import stem
 
class FingerPrint(object):
    def __init__(self):
        super(FingerPrint, self).__init__()
        self.remove_spl_char_regex = re.compile('[%s]' % re.escape(string.punctuation)) # regex to remove special characters
        self.remove_num = re.compile('[\d]+')
 
    def fp_steps(self,text):
        title = text.strip().lower()
        title_splchar_removed = self.remove_spl_char_regex.sub(" ",title)
        title_number_removed = self.remove_num.sub("", title_splchar_removed)
        words = title_number_removed.split()
        filter_stop_words = [w for w in words if not w in nltk.corpus.stopwords.words('english')]
        stemed = [stem(w) for w in filter_stop_words]
        return sorted(stemed)
     
    def fingerprint(self,text):
        fp = " ".join(self.fp_steps(text))
        return fp

In [ ]:
# map.py
import sys
import re
import string
from fingerprint import FingerPrint
 
f = FingerPrint()
 
for line in sys.stdin:
    try:
        print "%s\t%s" % (f.fingerprint(line),line.strip())
    except Exception as e:
        print e
        pass

In [ ]:
# reduce.py
 
import sys
from Levenshtein import distance
import json
 
DISTANCE = 20
cluster = {}
cid = 0
 
for i,line in enumerate(sys.stdin):
    cols = line.strip().split("\t")
    if i == 0:
        cluster[cid] = []
        cluster[cid].append(cols)
    else:
        last = cluster[cid][-1]
        if distance(last[0],cols[0]) <= DISTANCE:
            cluster[cid].append(cols)
        else:
            cid+=1
            cluster[cid] = []
            cluster[cid].append(cols)

            
for k,v in cluster.iteritems():
    print
    print "Cluster # ",k
    for entry in v:
        print entry[1]